# Import Package

In [39]:
import sys
 
# appending a path
sys.path.append('')
 
import os
import pandas as pd
import numpy as np
import pickle
# import SFR_PD_Recalibration_2023_Lib as tool
# import Concentra_SFR_Fit_for_Use_2024_Lib as lib
from matplotlib import pyplot as plt
import math

import Lib_RWA as Lib_RWA

del sys.modules['Lib_RWA']  # Remove the module from sys.modules to force a reload

import Lib_RWA as Lib_RWA

In [40]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\georgez\AppData\Local\Temp\ipykernel_18784\3764690850.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [41]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Inputs

In [42]:
snapshot_num = 202212 # 20250412 George commented out
snapshot = str(snapshot_num) # 20250412 George commented out
snapshot_date = '2022-12-31' # 20250412 George commented out

# snapshot_num = 202412 # 20250412 George added
# snapshot = str(snapshot_num) # 20250412 George added
# snapshot_date = '2024-12-31' # 20250412 George added


#CAR 2023 Ch5 prescribed
correlation_residential_mortgages = 0.15
correlation_residential_mortgages_rental = 0.22

CMHC_pd = 0.0001
CMHC_lgd = 0.11 #according to the newly developed Sovereign LGD model

lgd_gen_floor = 0.1 #general floor by CAR 2023


# Set Folders

In [43]:
# Get current working directory

current_dir = os.getcwd()  # 20250403 George added

# code_dir= "C:\\Users" +'\\' + username + '\\' + "Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\\code" # 20250403 George commented out Francesca/Joseph's code

code_dir = current_dir # 20250403 George added

if int(snapshot_num) == 202412:
    input_dir = code_dir + '\\..\\' + "Inputs" # 20250406 George added
    Output_dir = code_dir + '\\..\\' + "Outputs" #20250406 George added

if int(snapshot_num) == 202212:
    input_dir = code_dir + '\\..\\' + "Dec. 2022 RWA Inputs" #20250410 George added
    Output_dir = code_dir + '\\..\\' + "Replicated Dec 2022 Outputs" #20250410 George added

# Load PD Data

In [44]:
# Read PD Data

new_pd_data = pd.read_parquet(input_dir+ '\Concentra_PD_Scoring_2020-12-31_to_2024-12-31_wInsurer (from Francesca 20250411).parquet')  # 20250406 George added


# print columns in alphabetical order in the new_pd_data dataframe #20250402 George added

print('**************Before Renaming and add columns**************') #20250402 George added

print(sorted(new_pd_data.columns)) #20250402 George added





**************Before Renaming and add columns**************
['AIRB_scope_ind', 'Appraisal_Date_v2', 'Appraisal_Value_v2', 'Appraisal_YYYYMM', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Balance_Total', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_Orig', 'Beacon_Avg_App_CoApp_WOE', 'Beacon_Score_Current', 'Brookfield_Dwelling', 'City', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Concentra_Legacy_Ind', 'Concentra_ind', 'Current_Principal_Balance_CB', 'Days_in_Arrears', 'Delinquency_Status_Adj', 'Dwelling_Type', 'FSA', 'Foreclosure', 'Funded_Date', 'Inception_date', 'Insured_Ind', 'Insurer', 'IsREO', 'Loan_Number', 'MRS_Bin_PD', 'Metro_Region_BF_FMT', 'Mortgage_Number', 'Nb_entities_per_loan', 'Occupancy_Status', 'Original_Loan_Amount', 'Original_Loan_to_Value', 'PD_Post_MOC', 'PD_Pre_MOC', 'PD_scoring', 'Prior_24_Worse_Delinquent_Status', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_Delinquent_Status_FMT_Ad

In [45]:
# Remove Defaults
# Group by MRS_Bin, count distinct Mortgage_Number and min and max of PD_Post_MOC #20250402 George added
print('***************Before Removing Deault Loans*****************') #20250402 George added
print(new_pd_data.groupby(['MRS_Bin_PD'], as_index=False).agg({'Mortgage_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added

# Remove rows with PD_Post_MOC <=0 or >=1 (defaults) #20250402 George added

new_pd_data = new_pd_data.loc[(new_pd_data['PD_Post_MOC'] > 0) & (new_pd_data['PD_Post_MOC'] < 1)] #20250402 George added

# Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added

print('***************After Removing Deault Loans*****************') #20250402 George added

print(new_pd_data.groupby(['MRS_Bin_PD'], as_index=False).agg({'Mortgage_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added

***************Before Removing Deault Loans*****************
  MRS_Bin_PD Mortgage_Number PD_Post_MOC          
                     nunique         min       max
0        1.0           23096    0.002228  0.002228
1        2.0           15190    0.003622  0.003622
2        3.0            8616    0.007377  0.007377
3        4.0            5609    0.011499  0.011499
4        5.0            4817    0.019067  0.019067
5        6.0            4892    0.034554  0.034554
6        7.0            1650    0.107857  0.107857
7        8.0            1140    0.629118  0.629118
8        9.0             474    1.000000  1.000000
***************After Removing Deault Loans*****************
  MRS_Bin_PD Mortgage_Number PD_Post_MOC          
                     nunique         min       max
0        1.0           23096    0.002228  0.002228
1        2.0           15190    0.003622  0.003622
2        3.0            8616    0.007377  0.007377
3        4.0            5609    0.011499  0.011499
4        5.0

In [46]:
# print shape of new_pd_data #20250402 George added

print('***************Shape of new_pd_data*****************') #20250402 George added

print(new_pd_data.shape) #20250402 George added


# count values of islua_loan, loan_type, product_type and source_name

print('***************Count of islua_loan, loan_type, product_type and source_name*****************') #20250402 George added

print(new_pd_data.groupby(['islua_loan','loan_type', 'product_type', 'source_name'], as_index=False).agg({'Mortgage_Number': pd.Series.nunique})) #20250402 George added

***************Shape of new_pd_data*****************
(1073510, 67)
***************Count of islua_loan, loan_type, product_type and source_name*****************
   islua_loan    loan_type product_type              source_name  Mortgage_Number
0       False  residential        alt-a                     cmls              815
1       False  residential        alt-a  community trust company              223
2       False  residential        alt-a           first national             8555
3       False  residential        alt-a                     mcap             3057
4       False  residential        prime      energy credit union               59
5       False  residential        prime           first national             7713
6       False  residential        prime   frontline credit union               42
7       False  residential        prime                     mcap            15117
8       False  residential        prime        pace credit union               63


In [47]:
# Exclude Loans
#  If islua_loan!=1 AND loan_type = 'residential' AND Balance_CB > 0 AND product_type NOT IN ('heloc','bridge financing') AND source_name IN ('mcap','first national','cmls') THEN exclude_ind = 0, else exclude_ind = 1 #20250402 George added

new_pd_data['exclude_ind'] = np.where(
    (new_pd_data['islua_loan'] != 1) &
    (new_pd_data['loan_type'] == 'residential') &
    (new_pd_data['Current_Principal_Balance_CB'] > 0) &
    (~new_pd_data['product_type'].isin(['heloc', 'bridge financing'])) &
    (new_pd_data['source_name'].isin(['mcap', 'first national', 'cmls'])),
    0,
    1
) #20250402 George added

# Group by exclude_ind, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added

print('***************Count of exclude_ind*****************') #20250402 George added

print(new_pd_data.groupby(['exclude_ind'], as_index=False).agg({'Mortgage_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added

***************Count of exclude_ind*****************
  exclude_ind Mortgage_Number PD_Post_MOC          
                      nunique         min       max
0           0           35243    0.002228  0.629118
1           1             698    0.002228  0.629118


In [48]:
# Rename columns:  SL_Date -> Snapshot_Date, AIRB_PD_Risk_Rating -> MRS_Bin, Apprisal_Value -> Appraisal_Value_v2, Appraisla_Date -> Appraisal_Date_v2, occupancy_status -> Occupancy_Status, Balance_CB to Current_Principal_Balance_CB, 

# new_pd_data.rename(columns={'SL_Date':'Snapshot_Date', 'AIRB_PD_Risk_Rating':'MRS_Bin', 'Appraisal_Value':'Appraisal_Value_v2', 'Appraisal_Date':'Appraisal_Date_v2', 'occupancy_status':'Occupancy_Status', 'Balance_CB':'Current_Principal_Balance_CB'}, inplace=True)

# If columns PD_Post_MOC_Pre_Adj,def_90dpd_utp_filldown, exclude_ind  are missing, add them as NaN

if 'PD_Post_MOC_Pre_Adj' not in new_pd_data.columns: # 20250412 George added

    new_pd_data['PD_Post_MOC_Pre_Adj'] = np.nan  # 20250412 George added

if 'def_90dpd_utp_filldown' not in new_pd_data.columns: # 20250412 George added

    new_pd_data['def_90dpd_utp_filldown'] = np.nan  # 20250412 George added

if 'exclude_ind' not in new_pd_data.columns: # 20250412 George added

    new_pd_data['exclude_ind'] = 0  # 20250412 George added

# print columns in alphabetical order in the new_pd_data dataframe #20250402 George added
print('**************AFter Renaming and add columns**************') #20250402 George added

print(sorted(new_pd_data.columns)) #20250402 George added

**************AFter Renaming and add columns**************
['AIRB_scope_ind', 'Appraisal_Date_v2', 'Appraisal_Value_v2', 'Appraisal_YYYYMM', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Balance_Total', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_Orig', 'Beacon_Avg_App_CoApp_WOE', 'Beacon_Score_Current', 'Brookfield_Dwelling', 'City', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Concentra_Legacy_Ind', 'Concentra_ind', 'Current_Principal_Balance_CB', 'Days_in_Arrears', 'Delinquency_Status_Adj', 'Dwelling_Type', 'FSA', 'Foreclosure', 'Funded_Date', 'Inception_date', 'Insured_Ind', 'Insurer', 'IsREO', 'Loan_Number', 'MRS_Bin_PD', 'Metro_Region_BF_FMT', 'Mortgage_Number', 'Nb_entities_per_loan', 'Occupancy_Status', 'Original_Loan_Amount', 'Original_Loan_to_Value', 'PD_Post_MOC', 'PD_Post_MOC_Pre_Adj', 'PD_Pre_MOC', 'PD_scoring', 'Prior_24_Worse_Delinquent_Status', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_De

In [49]:
# Filter snapshot, exclusion and keep columns
new_pd_data_red = new_pd_data.loc[(new_pd_data['Snapshot_Date'] == snapshot_date)&(new_pd_data['PD_Post_MOC'].notna())&(new_pd_data['exclude_ind']==0),['Mortgage_Number','Snapshot_Date','PD_Pre_MOC','PD_Post_MOC_Pre_Adj','PD_Post_MOC','MRS_Bin_PD','Insured_Ind','Appraisal_Value_v2',
                                                             'Combo_LTV_Insured_Ind',  'Combo_Province_Metro_Override_WOE', 'def_90dpd_utp_filldown', 'Concentra_Legacy_Ind', 'Occupancy_Status','Original_Loan_to_Value', 'FSA','Appraisal_Date_v2','Current_Principal_Balance_CB','Insurer','Original_Loan_Amount','Original_Loan_to_Value']].rename(columns={'MRS_Bin':'MRS_Bin_PD'})

# change Mortgage_Number to string because later maturity data has Mortgage_Number as string (some Mortgage number in maturity starts with C as a string)

new_pd_data_red['Mortgage_Number'] = new_pd_data_red['Mortgage_Number'].astype(str) # 20250412 George added

In [50]:
new_pd_data_red.groupby(['PD_Pre_MOC','PD_Post_MOC_Pre_Adj','PD_Post_MOC','MRS_Bin_PD'],dropna=False).size()

PD_Pre_MOC  PD_Post_MOC_Pre_Adj  PD_Post_MOC  MRS_Bin_PD
0.001538    NaN                  0.002228     1.0           12446
0.002667    NaN                  0.003622     2.0            4402
0.005465    NaN                  0.007377     3.0            1847
0.008601    NaN                  0.011499     4.0            1042
0.014295    NaN                  0.019067     5.0             602
0.026508    NaN                  0.034554     6.0             573
0.081517    NaN                  0.107857     7.0             303
0.477554    NaN                  0.629118     8.0              74
dtype: int64

In [51]:
new_pd_data_red.Concentra_Legacy_Ind.value_counts()

0.0    19017
1.0     2272
Name: Concentra_Legacy_Ind, dtype: int64

In [52]:
# get Maturity - old Data strategy - Francesca did not use this code for Remaining_Term (instead LGD data has that column)
# import pyodbc
# server = 'EQSQLT01\RISK'  #Note that python is case sensitive, the server name has to be capital as they actually are
# database = 'Concentra'
# conn =pyodbc.connect(f'Driver={{SQL Server}}; Server={server};Database={database}')
# sql_query_loan = f'''
#     SELECT 
#         Mortgage_Number,
#         Remaining_Term
#        ,[Maturity_Date]
#        ,[Days_in_Arrears]
#     FROM 
#         Concentra.dbo.vw_CN_SFR_Loan_Dataset
#     WHERE
#         CONVERT(DATETIME, EOMONTH(SL_Date))='{snapshot_date}' and
#         IsLUA_Loan != 1 AND
#         Loan_Type = 'Residential' AND
#         Current_Principal_Balance_CB > 0 AND
#         Product_Type NOT IN ('HELOC','Bridge Financing') AND
#         Source_Name in ('MCAP','First National','CMLS')
# '''

# maturity = pd.read_sql_query(sql_query_loan,conn,parse_dates=['Maturity_Date'])

In [53]:
# maturity.Mortgage_Number=maturity.Mortgage_Number.str.lstrip('0')  # Again, the remaining_term is not used as this is old data strategy

# Load LGD Data

In [54]:
# new_lgd_data = pd.read_pickle(r"C:\Users\jasonz\OneDrive - Equitable Bank\Fit_for_Use Development\git_repos_MD\abhignanp\tempResults\Concentra_LGD_AllScored_Data.pkl") # 20250412 George commented out

new_lgd_data = pd.read_parquet(input_dir + '\Concentra_LGD_Scoring_2020-12-31_to_2024-12-31 (from Francesca 20250411).parquet')  # 20250406 George added


# change Mortgage_Number to string because later maturity data has Mortgage_Number as string (some Mortgage number in maturity starts with C as a string)

new_lgd_data['Mortgage_Number'] = new_lgd_data['Mortgage_Number'].astype(str) # 20250412 George added

new_lgd_data.loc[new_lgd_data['Remaining_Term']<0,'Remaining_Term']=0  #pasting Francesca's code to clean up Remaining_Term # 20250411 George added

In [55]:
# print min and max of Snapshot_Date in new_lgd_data #20250411 George added

print('***************Snapshot_Date in new_lgd_data*****************') #20250411 George added

print(new_lgd_data['Snapshot_Date'].min()) #20250411 George added

print(new_lgd_data['Snapshot_Date'].max()) #20250411 George added

***************Snapshot_Date in new_lgd_data*****************
2020-12-31 00:00:00
2024-12-31 00:00:00


# Merge PD and LGD Data

In [56]:
rwa_data=new_pd_data_red.merge(new_lgd_data[['Mortgage_Number','MRS_Bin','Pred_LGD','Mapped_LGD','Final_LGD','Remaining_Term']].rename(columns={'MRS_Bin':'MRS_Bin_LGD','Final_LGD':'Model_LGD'}), on ='Mortgage_Number', how='left')

In [57]:
# # Concentra_Remaining_Term=pd.read_excel(r'C:\Users\francescav\OneDrive - Equitable Bank\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\RWA\code\Inputs\Concentra Remaining Term December 2024 (from Francesca 20250404).xlsx')  #20250406 George commented out Francesca's code

# Concentra_Remaining_Term=pd.read_excel(input_dir + '\Concentra Remaining Term December 2024 (from Francesca 20250404).xlsx') #20250406 George added


# Concentra_Remaining_Term['Mortgage_Number'] = Concentra_Remaining_Term['Mortgage_Number'].astype(str) 
# Concentra_Remaining_Term.loc[Concentra_Remaining_Term['Remaining_Term']<0,'Remaining_Term']=0

In [58]:
rwa_data['Remaining_Term'].describe()

count    912136.000000
mean         26.162798
std          16.877722
min           0.000000
25%          11.000000
50%          24.000000
75%          40.000000
max          61.000000
Name: Remaining_Term, dtype: float64

In [59]:
rwa_data.Insurer.value_counts() # 20250403 FRAN added to verify below is correct - modified Cmhc: CMHC

Canada Guaranty    354426
CMHC               313687
Conventional       244023
Name: Insurer, dtype: int64

# Map Insurer to Insured_class

In [60]:
insured_class = {'Sagen':'Sagen', 'Conventional':'Uninsured', 'Canada Guaranty':'CG', 'CMHC Portfolio':'CMHC',
       'Cmhc':'CMHC', 'Sagen Portfolio':'Sagen', 'Canada Guaranty Portfolio':'CG'}

# if Insurer is one of the keys in the dictionary, then assign the value to the Insured_class column, otherwise assign the original value of Insurer to the Insured_class column

rwa_data['Insured_class'] = rwa_data['Insurer'].apply(lambda x: insured_class[x] if x in insured_class else x) # 20250411 George added

# rwa_data['Insured_class']= rwa_data['Insurer'].map(insured_class) # 20250411 George commented out because CMHC was not in the dictionary and changed to missing if not in the dictionary

# assign missing values in Insured_class to Uninsured # 20250411 George added

rwa_data['Insured_class'].fillna('Uninsured', inplace=True) # 20250411 George added

# rwa_data['corr_uninsured'] =rwa_data['Occupancy_Status'].map(lambda x:0.22 if x in ['Rented','Unknown'] else 0.15) # 20250412 George commented out
rwa_data['corr_uninsured'] =rwa_data['Occupancy_Status'].map(lambda x:0.15 if x in ['owner occupied','Owner Occupied','Owner_Occupied','owner_occupied'] else 0.22) # 20250412 George added

In [61]:
rwa_data.columns.values

array(['Mortgage_Number', 'Snapshot_Date', 'PD_Pre_MOC',
       'PD_Post_MOC_Pre_Adj', 'PD_Post_MOC', 'MRS_Bin_PD', 'Insured_Ind',
       'Appraisal_Value_v2', 'Combo_LTV_Insured_Ind',
       'Combo_Province_Metro_Override_WOE', 'def_90dpd_utp_filldown',
       'Concentra_Legacy_Ind', 'Occupancy_Status',
       'Original_Loan_to_Value', 'FSA', 'Appraisal_Date_v2',
       'Current_Principal_Balance_CB', 'Insurer', 'Original_Loan_Amount',
       'Original_Loan_to_Value', 'MRS_Bin_LGD', 'Pred_LGD', 'Mapped_LGD',
       'Model_LGD', 'Remaining_Term', 'Insured_class', 'corr_uninsured'],
      dtype=object)

# Add columns 

In [62]:
# dt_adjust={1:0.016497,2:0.040137,3:0.104959,4:0.180642}   #20250412 George commented out the numbers are not the latest and this will not be used in the final result
# rwa_data['LGD_DT_Adjusted']=rwa_data['MRS_Bin_LGD'].map(dt_adjust) #20250412 George commented out the numbers are not the latest and this will not be used in the final result

# if LGD_DT_Adjusted is not in the columns, add it as NaN

if 'LGD_DT_Adjusted' not in rwa_data.columns: # 20250412 George added
    rwa_data['LGD_DT_Adjusted'] = np.nan  # 20250412 George added, the numbers are not the latest and this will not be used in the final result

rwa_data['Segment_Avg_LGD'] = rwa_data.loc[:,'Mapped_LGD']
rwa_data['Base_Line_LGD'] =rwa_data.loc[:,'Mapped_LGD']
rwa_data['Advance_Amount'] = rwa_data.loc[:,'Original_Loan_Amount']
rwa_data['Years_to_maturity'] = rwa_data.loc[:,'Remaining_Term']/12
rwa_data['EAD'] = rwa_data.loc[:,'Current_Principal_Balance_CB']

# RWA with 0 Addon

In [63]:
# Force Addon to be 0

rwa_raw_data_5 = rwa_data.copy()
rwa_raw_data_5['AddOn']=0


In [64]:
# Run RWA calculation function

[df_out, rwa_by_Insured_class, rwa_by_MRS_Bin, res_data]  = Lib_RWA.rwa_calculation(rwa_raw_data_5, lgd_gen_floor, CMHC_lgd, CMHC_pd)


# #################################################################
# ####################### RWA Caculation #########################
# #################################################################

# df_out=pd.DataFrame()

# # record result data 
# res_data=pd.DataFrame()


# for ind_contol in range(0,3):
#     print(ind_contol)

#     # baseline LGD, Baseline PD
#     if ind_contol==0:
#         rwa_raw_data = rwa_raw_data_5.copy().drop(['PD_Post_MOC','PD_Post_MOC_Pre_Adj'], axis=1).rename({'PD_Pre_MOC': 'CalibratedPD', 'Base_Line_LGD' : 'Pre_final_LGD'  }, axis='columns'); 
#     #  PD_Post_MOC, LGD_DT_JUST     
#     if ind_contol==1:
#         rwa_raw_data = rwa_raw_data_5.copy().drop(['PD_Pre_MOC','PD_Post_MOC_Pre_Adj'], axis=1).rename({'PD_Post_MOC': 'CalibratedPD', 'LGD_DT_Adjusted': 'Pre_final_LGD' }, axis='columns'); 
#     #  PD_Post_MOC, Final LGD     
#     if ind_contol==2:
#         rwa_raw_data = rwa_raw_data_5.copy().drop(['PD_Pre_MOC','PD_Post_MOC_Pre_Adj'], axis=1).rename({'PD_Post_MOC': 'CalibratedPD' , 'Model_LGD': 'Pre_final_LGD' }, axis='columns'); 
        
#     #  aseline LGD,  PD after long-run adjustment
#     '''
#     if ind_contol==3:
#         rwa_raw_data = rwa_raw_data_5.copy().drop(['PD_Post_MOC','PD_Post_MOC_Pre_Adj'], axis=1).rename({'PD_Pre_MOC': 'CalibratedPD', 'Base_Line_LGD' : 'Pre_final_LGD'  }, axis='columns'); 
        
#         adj_Longrun = pd.read_excel(os.path.join(adhoc_path, 'MOC_Summary.xlsx'), sheet_name= 'Sheet1') [['Adj_Longrun']].values[0].item()
#         rwa_raw_data['CalibratedPD'] =  rwa_raw_data['CalibratedPD']*(1+adj_Longrun)
#     '''
    
#     # Apply regulatory floors and derive the final LGD
#     rwa_raw_data['DLGD_floor'] = rwa_raw_data['Segment_Avg_LGD'] + rwa_raw_data['AddOn']
    
#     # sum(~rwa_raw_data['DLGD_floor'].isna())
#     # sum((~rwa_raw_data['Segment_Avg_LGD'].isna()) & (rwa_raw_data.Insured_class.isin(["CMHC","Sagen", "CG"])))
    
#     # sum(~rwa_raw_data['Pre_final_LGD'].isna())
#     # sum(~rwa_raw_data['Segment_Avg_LGD'].isna() )
#     # sum(~(rwa_raw_data.Insured_class.isin(["CMHC","Sagen", "CG"]) ))

#     rwa_raw_data.loc[~rwa_raw_data['Pre_final_LGD'].isna(),'OSFI_LGD_floor'] = lgd_gen_floor
#     rwa_raw_data['Final_LGD'] = rwa_raw_data[['Pre_final_LGD','DLGD_floor','OSFI_LGD_floor']].max(axis='columns')

#     # sum(~rwa_raw_data['Final_LGD'].isna())
    
#     #for insured loans which do not have Gen3 LGD, previous Final LGD is used
#     #rwa_raw_data.loc[rwa_raw_data['Pre_final_LGD'].isna(),'Final_LGD'] = rwa_raw_data['Final_LGD_old']
    
    
#     ## CG and Sagen use 11% LGD 
#     rwa_raw_data.loc[rwa_raw_data.Insured_class.isin(["CMHC","Sagen", "CG"]), 'Final_LGD' ] = CMHC_lgd

#     # sum(rwa_raw_data['Final_LGD'].isna())
    
    
#     ## insured_PMI_ratio 

    
#     rwa_raw_data["deductible_amount"] = 0.1 * rwa_raw_data.Advance_Amount
#     rwa_raw_data["insured_PMI_ratio"] = np.where(rwa_raw_data.EAD > 0, (rwa_raw_data.EAD - rwa_raw_data.deductible_amount) / rwa_raw_data.EAD, 0)
    
#     # !!!! can be negative; When insured_PMI_ratio is negative it will be handle in cond_PMI_dec case.
    
    
#   #  sum( round(rwa_raw_data["insured_PMI_ratio"]*10**5) == round(rwa_raw_data["insured_PMI_ratio_old"] *10**5) ) #56478
    
#     ## Generate indicators 
#     cond_CMHC = (rwa_raw_data.Insured_class.isin(['CMHC']))
    
#     cond_PMI = (rwa_raw_data.Insured_class.isin(["Sagen", "CG"])) & (rwa_raw_data.EAD > 0.1 * rwa_raw_data.Advance_Amount)

#     cond_PMI_dec = (rwa_raw_data.Insured_class.isin(["Sagen", "CG"])) & (rwa_raw_data.EAD <= 0.1 * rwa_raw_data.Advance_Amount)
    
#     cond_UI = (rwa_raw_data.Insured_class.isin(["Uninsured"]))
    
    
#     ## Maturity adjustment
    
#     def func_maturity_adj(pd_value):
#         import numpy as np
#         adj = (0.11852 - 0.05478 * np.log(pd_value)) ** 2
#         return adj  ##CAR 2023 5.3.1(i) 66.
    
#     rwa_raw_data.loc[cond_UI | cond_PMI_dec, "Maturity_adj"] = rwa_raw_data.loc[:, ['CalibratedPD']].apply(lambda s: func_maturity_adj(s[0]), axis=1) #not actually used since retail RWA formula will be applied
    
#     rwa_raw_data.loc[cond_CMHC | cond_PMI, "Maturity_adj"] = func_maturity_adj(CMHC_pd)
    

#     ## 'corr_uninsured'  depends on rental income indicator and has two values correlation_residential_mortgages_rental,correlation_residential_mortgages
    
    
#     # correlation for CMHC and insured deductible for PMI ?
#     def correlation_corp(pd_value):
#         import numpy as np
#         #     corr =  round(0.12 * (1 - np.exp(-50 * pd_value)) / (1- np.exp(-50)) +
#         #                       0.24 * (1 - (1- np.exp(-50 * pd_value))/(1 - np.exp(-50))),6) ##CAR 2023 5.3.1(i) 66.
#         corr = 0.12 * (1 - np.exp(-50 * pd_value)) / (1 - np.exp(-50)) + 0.24 * (
#                 1 - (1 - np.exp(-50 * pd_value)) / (1 - np.exp(-50)))
#         return corr
    
#     rwa_raw_data.loc[cond_CMHC | cond_PMI, 'corr_insured'] = correlation_corp(CMHC_pd)
    
    
#     ## func_risk_weight_retail 
#     def func_risk_weight_retail(pd_value, lgd_value, corr):  # CAR 2023 Chapter 5 paragrah 81
#         import scipy.stats as st
#         pd_value = float(pd_value)
#         lgd_value = float(lgd_value)
#         corr = float(corr)
#         pd_UL = st.norm.cdf(
#             (1 - corr) ** (-0.5) * st.norm.ppf(pd_value) + (corr / (1 - corr)) ** (0.5) * st.norm.ppf(0.999))
#         K = pd_UL * lgd_value - pd_value * lgd_value
#         # return  round(pd_UL * lgd_value - pd_value * lgd_value,6)
#         return K
    
    
#     def func_risk_weight_corp(pd_value, lgd_value, corr, maturity_adj,
#                               effective_maturity):  # CAR 2023 Chapter 5 paragraph 66
#         import scipy.stats as st
#         a = (lgd_value * st.norm.cdf((1 - corr) ** (-0.5) * st.norm.ppf(pd_value) +
#                                      (corr / (1 - corr)) ** 0.5 * st.norm.ppf(0.999)))
#         tail_value = a - (pd_value * lgd_value)
#         adj = (1 + (effective_maturity - 2.5) * maturity_adj) / (1 - (1.5 * maturity_adj))
#         k = tail_value * adj
#         # return  round(tail_value * adj,6)
#         return k
    
    
#     # retail risk weight function for uninsured loans
#     rwa_raw_data['risk_weight_uninsured'] = 0
    
#     rwa_raw_data.loc[cond_UI, 'risk_weight_uninsured'] = rwa_raw_data.loc[cond_UI, ['CalibratedPD', 'Final_LGD', 'corr_uninsured']].apply(
#             lambda s: func_risk_weight_retail(s[0], s[1], s[2]), axis=1)
    
    
#     # re-write uninsured deductible for PMI using borrower PD and 100% LGD and retail risk weight function
#     # df["LGD_uninsured"]=1.0
    
#     rwa_raw_data.loc[cond_PMI, 'risk_weight_uninsured'] = rwa_raw_data.loc[cond_PMI, ['CalibratedPD', 'corr_uninsured']].apply(
#             lambda s: func_risk_weight_retail(s[0], 1, s[1]), axis=1)
    
#     rwa_raw_data.loc[cond_PMI_dec, 'risk_weight_uninsured'] = rwa_raw_data.loc[cond_PMI_dec, ['CalibratedPD', 'corr_uninsured']].apply(
#             lambda s: func_risk_weight_retail(s[0], 1, s[1]), axis=1)
    
        
#     # insured CMHC including PMI insured backstop use non-retail risk function
#     rwa_raw_data['risk_weight_insured'] = 0
    
#     rwa_raw_data.loc[cond_CMHC, 'risk_weight_insured'] = rwa_raw_data.loc[cond_CMHC, ['Final_LGD', 'corr_insured','Maturity_adj', 'Years_to_maturity']].apply(
#          lambda s: func_risk_weight_corp(CMHC_pd, CMHC_lgd, s[1], s[2], s[3]), axis=1)
    
    
    
#     rwa_raw_data.loc[cond_PMI, 'risk_weight_insured'] = rwa_raw_data.loc[cond_PMI, ['Final_LGD', 'corr_insured','Maturity_adj', 'Years_to_maturity']].apply(
#          lambda s: func_risk_weight_corp(CMHC_pd,  CMHC_lgd, s[1], s[2], s[3]), axis=1)
    
    
    
#     ## RWA_res_mortgage 
#     def RWA_res_mortgage(capital_value, capital_value_insured, EAD_value, insured_ratio):
#         # insured_ratio: Ratio of insured
#         #     insured_RWA = round(capital_value_insured * 12.5 * EAD_value,0)
#         #     uninsured_RWA = round(capital_value * 12.5 * EAD_value,0)
#         insured_RWA = capital_value_insured * 12.5 * EAD_value
#         uninsured_RWA = capital_value * 12.5 * EAD_value
#         rwa_total = insured_ratio * insured_RWA + (1 - insured_ratio) * uninsured_RWA
#         return rwa_total
    
    
    
#     # 1. CMHC insured
#     rwa_raw_data.loc[cond_CMHC, 'RWA_new'] = rwa_raw_data.loc[cond_CMHC, [ 'risk_weight_insured', 'EAD']].apply(lambda s: RWA_res_mortgage(0, s[0], s[1], 1), axis=1)
    
    
      
#     # 2. Sagen and CG
#     rwa_raw_data.loc[cond_PMI, 'RWA_new'] = rwa_raw_data.loc[
#           cond_PMI, ['risk_weight_uninsured', 'risk_weight_insured', 'EAD', "insured_PMI_ratio"]].apply(lambda s: RWA_res_mortgage(s[0], s[1], s[2], s[3]), axis=1)

#     rwa_raw_data.loc[cond_PMI_dec, 'RWA_new'] = rwa_raw_data.loc[cond_PMI_dec, ['risk_weight_uninsured', 'EAD']].apply(
#           lambda s: RWA_res_mortgage(s[0], 0, s[1], 0), axis=1)
    
#     # 3. Uninsured Loans
#     rwa_raw_data.loc[cond_UI, 'RWA_new'] = rwa_raw_data.loc[cond_UI, ['risk_weight_uninsured', 'EAD']].apply(
#           lambda s: RWA_res_mortgage(s[0], 0, s[1], 0), axis=1)
    
#     #test = rwa_raw_data.loc[ (round(rwa_raw_data["rwa_airb"]*10**6) != round(rwa_raw_data["RWA_new"]*10**6) )  &  (rwa_raw_data["Insured_class"] == 'CMHC') ][['Insured_class',"rwa_airb",'RWA_new']]
    
#     #rwa_raw_data.loc[rwa_raw_data['Insured_class'] =='Uninsufred', 'RWA_new'].sum()
#     ## Summary table
    
#     if ind_contol==0:
#       df_out["EAD"]= [rwa_raw_data["EAD"].sum()]
#       # df_out["SA RWA"]= [rwa_raw_data["RWA_standardized"].sum()]  
#       # df_out["In Product RWA"]= [rwa_raw_data["rwa_airb"].sum()]  --gen 2 
#       df_out["Pre MOC RWA"]= [rwa_raw_data["RWA_new"].sum()]
      
#       df_out["Pre MOC RWA (uninsured)"]= [rwa_raw_data.loc[rwa_raw_data['Insured_class']=='Uninsured']["RWA_new"].sum()]
#       df_out["EAD (uninsured)"]= [rwa_raw_data.loc[rwa_raw_data['Insured_class']=='Uninsured']["EAD"].sum()]

      
#       # rwa_airb_by_Insured_class = rwa_raw_data.groupby(['Insured_class' ],as_index=False)[['EAD','rwa_airb','RWA_standardized']].sum().rename({'rwa_airb': 'rwa_airb_by_Insured_class', 'RWA_standardized': 'RWA_standardized_by_Insured_class'}, axis='columns')
      
#       rwa_mapped_by_Insured_class = rwa_raw_data.groupby(['Insured_class' ],as_index=False)[['EAD','RWA_new']].sum().rename({'RWA_new': 'rwa_mapped_by_Insured_class'}, axis='columns')
      
#       # rwa_airb_by_MRS_Bin = rwa_raw_data.groupby(['MRS_Bin_PD' ],as_index=False)[['EAD','rwa_airb','RWA_standardized']].sum().rename({'rwa_airb': 'rwa_airb_by_MRS_Bin'}, axis='columns')
      
#       rwa_mapped_by_MRS_Bin = rwa_raw_data.groupby(['MRS_Bin_PD' ],as_index=False)[['EAD','RWA_new']].sum().rename({'RWA_new': 'rwa_mapped_by_MRS_Bin'#,'RWA_standardized': 'RWA_standardized_by_MRS_Bin'
#                                                                                                            }, axis='columns')
      
#       # rwa_by_Insured_class0 =pd.merge(rwa_airb_by_Insured_class, rwa_mapped_by_Insured_class, how='left',on=['Insured_class'] )
      
#       # rwa_by_MRS_Bin0 =pd.merge(rwa_airb_by_MRS_Bin, rwa_mapped_by_MRS_Bin, how='left',on=['MRS_Bin_PD'] )
      
#     if ind_contol==1: 
#         df_out["Post MOC Pre LR RWA"]= [rwa_raw_data["RWA_new"].sum()]
#         df_out["Post MOC Pre LR RWA (uninsured)"]= [rwa_raw_data.loc[rwa_raw_data['Insured_class']=='Uninsured']["RWA_new"].sum()]
    
#     '''   
#     if ind_contol==3: 
#         df_out["Post PD longrun"]= [rwa_raw_data["RWA_new"].sum()]
#         df_out["Post PD longrun (uninsured)"]= [rwa_raw_data.loc[rwa_raw_data['Insured_class']=='Uninsured']["RWA_new"].sum()]    
#     ''' 
        
#     if ind_contol==2: 
#         df_out["Final RWA"]= [rwa_raw_data["RWA_new"].sum()]

#         rwa_Post_MOC_by_Insured_class = rwa_raw_data.groupby(['Insured_class' ],as_index=False)['RWA_new'].sum().rename({'RWA_new': 'rwa_Post_MOC_by_Insured_class'}, axis='columns')

#         rwa_Post_MOC_by_MRS_Bin = rwa_raw_data.groupby(['MRS_Bin_PD'],as_index=False)['RWA_new'].sum().rename({'RWA_new': 'rwa_Post_MOC_by_MRS_Bin'}, axis='columns')

#         # rwa_by_Insured_class =pd.merge(rwa_by_Insured_class0, rwa_Post_MOC_by_Insured_class, how='left',on=['Insured_class'] )
#         rwa_by_Insured_class =pd.merge( rwa_mapped_by_Insured_class, rwa_Post_MOC_by_Insured_class, how='left',on=['Insured_class'] )

#         # rwa_by_MRS_Bin =pd.merge(rwa_by_MRS_Bin0, rwa_Post_MOC_by_MRS_Bin, how='left',on=['MRS_Bin_PD'] )
#         rwa_by_MRS_Bin =pd.merge(rwa_mapped_by_MRS_Bin, rwa_Post_MOC_by_MRS_Bin, how='left',on=['MRS_Bin_PD'] )

#         #copy to res_data
#         res_data = rwa_raw_data.copy()

0
1
2


In [65]:
# with pd.ExcelWriter(
#   "C:\\Users\\jasonz\\OneDrive - Equitable Bank\\Fit_for_Use Development\\RWA\\eqb_rwa_addon_zero.xlsx",
#       mode="a",
#       engine="openpyxl",
#       if_sheet_exists="replace",
#   ) as writer:
#     res_data.to_excel(writer, sheet_name="RWA_Data") 


In [66]:
# Save Data
# 20250403 Fran - Added to save concentra - renamed to Concentra
# with pd.ExcelWriter(
#    r'C:\Users\francescav\OneDrive - Equitable Bank\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\RWA\code\Outputs\Concentra_rwa_addon_zero.xlsx') as writer:  #20250406 George commented out Francesca's code
     
with pd.ExcelWriter(
   Output_dir + '\concentra_rwa_addon_zero.xlsx') as writer: #20250406 George added
          
     res_data.to_excel(writer, sheet_name="RWA_Data")

# Borrow Add on Impact from EQB

In [67]:
# We need to add a cross table of lgd_bins and PD bins for rwa with add-on. However, we don't have account level addon info. therefore we take the impact from EQB at the Combo_LTV_Insured_Ind X Combo_Province_Metro_Override_WOE level

# addon_impact =pd.read_excel(r"C:\Users\francescav\OneDrive - Equitable Bank\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\RWA\code\Outputs\addon_impact.xlsx")  # 20250406 George commented out Francesca's code

addon_impact =pd.read_excel(Output_dir+r"\addon_impact.xlsx") # 20250406 George added

rwa_raw_data = res_data.copy() # 20250411 George added because a new res_data is created in the above function

In [68]:
# 20250403 - Francesca Add on already has mapped values !
addon_impact.Combo_Province_Metro_Override_WOE.value_counts()

 0.165650    11
-0.389936    10
 0.253031     9
-0.161658     8
Name: Combo_Province_Metro_Override_WOE, dtype: int64

In [69]:
addon_impact.Combo_Province_Metro_Override_WOE.fillna(0, inplace=True)
addon_impact.Combo_Province_Metro_Override_WOE =addon_impact.Combo_Province_Metro_Override_WOE.round(8)

In [70]:
addon_impact.rename(columns={'impact':'addon_impact'},inplace=True)

In [71]:
#process to make sure the merge works;
rwa_raw_data.Combo_Province_Metro_Override_WOE=rwa_raw_data.Combo_Province_Metro_Override_WOE.round(6).fillna(0)
addon_impact.Combo_Province_Metro_Override_WOE = addon_impact.Combo_Province_Metro_Override_WOE.round(6).fillna(0)
addon_impact.Combo_LTV_Insured_Ind=addon_impact.Combo_LTV_Insured_Ind.fillna('')

In [72]:
rwa_raw_data_addon=rwa_raw_data.merge(addon_impact[['Combo_LTV_Insured_Ind','Combo_Province_Metro_Override_WOE','addon_impact']], on=['Combo_LTV_Insured_Ind','Combo_Province_Metro_Override_WOE'],how='left')

In [73]:
rwa_raw_data_addon.addon_impact = rwa_raw_data_addon.addon_impact.fillna(1)

In [74]:
rwa_raw_data_addon['RWA_AddOn']=rwa_raw_data_addon['RWA_new']*rwa_raw_data_addon['addon_impact']

# Save Data

In [75]:

#with open(r'C:\Users\jasonz\OneDrive - Equitable Bank\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\RWA\code\Concentra_RWA_202410.pkl', 'wb') as file: 
      
    # A new file will be created 
    #pickle.dump(rwa_raw_data_addon, file) 

# with open(r'C:\Users\francescav\OneDrive - Equitable Bank\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\RWA\code\Outputs\Concentra_RWA_202412.pkl', 'wb') as file:   # 20250406 George commented out Francesca's code

with open( Output_dir +r'\concentra_rwa_addon.pkl', 'wb') as file: # 20250406 George added

    # A new file will be created 
    pickle.dump(rwa_raw_data_addon, file) 

In [76]:
#Save an excel copy

rwa_raw_data_addon.to_excel(Output_dir + r'\concentra_rwa_addon.xlsx')  #20250406 George added